## Small Oscillations Double Pendulum

##Preamble

In [24]:
from sympy import symbols, Function, sin, cos, Rational, diff, simplify, zeros, expand, Transpose, sqrt, Matrix

Symbolic variables

In [2]:
t,l1,l2,m1,m2,g=symbols('t,l1,l2,m1,m2,g',positive=True)

Generalized coordinates

In [3]:
phi1,phi2=Function('phi1')(t),Function('phi2')(t)

Cartesian Coordinates

In [4]:
x1=l1*sin(phi1)
x2=l1*sin(phi1)+l2*sin(phi2)
y1=l1*cos(phi1)
y2=l1*cos(phi1)+l2*cos(phi2)

## Kinetic Energy $T$

In [5]:
T=(Rational(1,2)*m1*((x1.diff(t))**2+(y1.diff(t))**2)+Rational(1,2)*m2*((x2.diff(t))**2+(y2.diff(t))**2)).simplify()
T

l1**2*m1*Derivative(phi1(t), t)**2/2 + m2*(l1**2*Derivative(phi1(t), t)**2 + 2*l1*l2*cos(phi1(t) - phi2(t))*Derivative(phi1(t), t)*Derivative(phi2(t), t) + l2**2*Derivative(phi2(t), t)**2)/2

## Potential Energy $U$ (notice that $y$ increases downwards)

In [6]:
U=(-m1*g*y1-m2*g*y2).simplify()
U

-g*(l1*m1*cos(phi1(t)) + l1*m2*cos(phi1(t)) + l2*m2*cos(phi2(t)))

## Lagrangian $L=T-U$

In [7]:
Lag=(T-U).simplify()
Lag

g*(l1*m1*cos(phi1(t)) + l1*m2*cos(phi1(t)) + l2*m2*cos(phi2(t))) + l1**2*m1*Derivative(phi1(t), t)**2/2 + m2*(l1**2*Derivative(phi1(t), t)**2 + 2*l1*l2*cos(phi1(t) - phi2(t))*Derivative(phi1(t), t)*Derivative(phi2(t), t) + l2**2*Derivative(phi2(t), t)**2)/2

## For concretness, in the following we will focus on a double pendulum with $l_1=l$, $l_2=l$, $m_1=m$, $m_2=m$

In [8]:
l,m=symbols('l,m',positive=True)

## Inertia Tensor

In [9]:
dot_phi=(phi1.diff(t),phi2.diff(t))
M0=zeros(len(dot_phi))
for i in range(len(dot_phi)):
  for j in range(len(dot_phi)):
    M0[i,j]=simplify((T.diff(dot_phi[i])).diff(dot_phi[j]).subs([(phi1,0),(phi2,0)]))
M0

Matrix([
[l1**2*(m1 + m2), l1*l2*m2],
[       l1*l2*m2, l2**2*m2]])

In [10]:
M=simplify(M0.subs([(l1,l),(l2,l),(m1,m),(m2,m)]))
M

Matrix([
[2*l**2*m, l**2*m],
[  l**2*m, l**2*m]])

## Harmonic Tensor

In [11]:
phi=(phi1,phi2)
K0=zeros(len(phi))
for i in range(len(phi)):
  for j in range(len(phi)):
    K0[i,j]=simplify((U.diff(phi[i])).diff(phi[j]).subs([(phi1,0),(phi2,0)]))
K0

Matrix([
[g*l1*(m1 + m2),       0],
[             0, g*l2*m2]])

In [12]:
K=simplify(K0.subs([(l1,l),(l2,l),(m1,m),(m2,m)]))
K

Matrix([
[2*g*l*m,     0],
[      0, g*l*m]])

In this case, the harmonic tensor is alredy diagonal. Thus we can use it to perform the scale transformation that reshapes it into the unit matrix. In turn, the same transformation applied to the inertia tensor is the following:

In [13]:
MK=simplify(sqrt(K)**(-1)*M*sqrt(K)**(-1))
MK

Matrix([
[            l/g, sqrt(2)*l/(2*g)],
[sqrt(2)*l/(2*g),             l/g]])

The method **diagonalize** returns the modal matrix $T$ thad diagonalizes the input matrix $MK$ rendering a diagonal matrix $M_{KD}$ according to $T^{-1}M_KT=M_{KD}$

In [14]:
T,MKD=MK.diagonalize()

Orthonormalization of the eigenvectors in the modal matrix $T$ that produces an orthogonal modal matrix $O_{M}$ such that $O_{MK}^T M_K O_{MK}=M_{KD}$

In [15]:
OMK=simplify(T*(sqrt((Transpose(T)*T)**(-1))))
OMK

Matrix([
[-sqrt(2)/2, sqrt(2)/2],
[ sqrt(2)/2, sqrt(2)/2]])

Verification of orthonormality and diagonalization

In [16]:
simplify(Transpose(OMK)*OMK)

Matrix([
[1, 0],
[0, 1]])

In [17]:
simplify(Transpose(OMK)*MK*OMK-MKD)

Matrix([
[0, 0],
[0, 0]])

Finally the diagonal matrix of squared eigenfrequencies is the inverse of $M_{KD}$

In [18]:
WD=(MKD)**(-1)
WD

Matrix([
[g/(l*(1 - sqrt(2)/2)),                     0],
[                    0, g/(l*(sqrt(2)/2 + 1))]])

Modal Matrix

In [19]:
S=simplify(sqrt(K)**(-1)*OMK*sqrt(MKD)**(-1))
S

Matrix([
[-sqrt(2)/(2*l*sqrt(m)*sqrt(2 - sqrt(2))), sqrt(2)/(2*l*sqrt(m)*sqrt(sqrt(2) + 2))],
[         1/(l*sqrt(m)*sqrt(2 - sqrt(2))),         1/(l*sqrt(m)*sqrt(sqrt(2) + 2))]])

Normal coordinates

In [21]:
((S)**(-1))*Matrix([[phi1], [phi2]])

Matrix([
[(-sqrt(2)*l*sqrt(m) + l*sqrt(m))*phi1(t)/sqrt(2 - sqrt(2)) + (-sqrt(2)*l*sqrt(m) + 2*l*sqrt(m))*phi2(t)/(2*sqrt(2 - sqrt(2)))],
[                                        sqrt(2)*l*sqrt(m)*sqrt(sqrt(2) + 2)*phi1(t)/2 + l*sqrt(m)*sqrt(sqrt(2) + 2)*phi2(t)/2]])

## General solution for the angles

In [29]:
C1,C2,delta1,delta2,Delta=symbols('C1,C2,delta1,delta2,Delta')

In [30]:
phi=S*Matrix([[C1*cos(WD[0]*t+delta1)], [C1*cos(WD[0]*t+delta2)]])
phi

Matrix([
[-sqrt(2)*C1*cos(delta1 + g*t/(l*(1 - sqrt(2)/2)))/(2*l*sqrt(m)*sqrt(2 - sqrt(2))) + sqrt(2)*C1*cos(delta2 + g*t/(l*(1 - sqrt(2)/2)))/(2*l*sqrt(m)*sqrt(sqrt(2) + 2))],
[                     C1*cos(delta1 + g*t/(l*(1 - sqrt(2)/2)))/(l*sqrt(m)*sqrt(2 - sqrt(2))) + C1*cos(delta2 + g*t/(l*(1 - sqrt(2)/2)))/(l*sqrt(m)*sqrt(sqrt(2) + 2))]])